# DattoRMM - API - Patching

In [ ]:
import requests
import json
import pandas as pd

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')

In [ ]:
import requests
from requests.structures import CaseInsensitiveDict

# import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']

## Create Auth Token

In [ ]:
# call token api url
token_uri = config['dattormm']['token_uri']

# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create All Devices DataFrame

In [ ]:
# request content response
request_url = f'{base_uri}/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}' 
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_nextpage = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices,df_nextpage],ignore_index=False)

In [ ]:
df_devices

In [ ]:
df_manage = df_devices[['uid','hostname','patchManagement']]

In [ ]:
for k,v in df_manage['patchManagement'].items():
    print(k,v)

## Create Construct Expanded DataFrame

In [ ]:
device_uid = '[[REDACTED_UUID]]'
component_id = '[REDACTED_UUID]'

In [ ]:
# request content response
request_url = 'https://concord-api.centrastage.net/api/v2/device/[[REDACTED_UUID]]/alerts/resolved'
#request_url = f'{base_uri}/job'
# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = CaseInsensitiveDict()
data['jobComponent'] = '[REDACTED_UUID] '

#print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict)
device_info_json = c_dict